In [12]:
import numpy as np
import pandas as pd
import datetime

# work_time_table = pd.read_csv('./근무시간_테이블.csv')

In [61]:
class working_time_calculator:
    def __init__(self):
#         self.work_time_table = pd.read_csv(file_path)
#         self.work_time_table = self.work_time_table.sort_values(['date'], reset_index = True)
        self.work_time_table = pd.DataFrame(columns = ['date', 'month', 'week_num', 'week_day', 'start_time', 'end_time', 'spend_time', 'rest_time'])
        self.rest_time = 1
        
        self.days_dic = {
                            1: '월요일',
                            2: '화요일',
                            3: '수요일',
                            4: '목요일',
                            5: '금요일',
                            6: '토요일',
                            7: '일요일'
        }
        
        return
    
    
    # 현재 시점 기준으로 업무 시작, 종료를 입력하는 함수
    def work_start(self, start_time = None):
        if start_time == None:
            start_time = datetime.datetime.now()
            
        else:
            start_time = datetime.datetime.fromisoformat(start_time)
            
        date = datetime.datetime.strftime(start_time, '%Y%m%d')
        month = datetime.datetime.strftime(start_time, '%Y%m')
        week_num = start_time.isocalendar()[1]
        week_day = self.days_dic[start_time.isoweekday()]
        start_time = datetime.datetime.strftime(start_time, '%Y-%m-%d %H:%M')
        
        # 해당 일자의 데이터가 있을 경우, 없을 경우로 구분
        # 없을 경우 - 신규 데이터 입력
        # 없을 경우 - 기존 출근 시간을 수정
        update_state = not self.work_time_table.loc[self.work_time_table['date'] == date].empty
        
        if update_state:
            update_row = pd.DataFrame([[date, month, week_num, week_day, start_time]],
                                      columns = ['date', 'month', 'week_num', 'week_day', 'start_time'])
            self.work_time_table.update(update_row)
            
            # end_time까지 입력되어 있는 경우라면 spend_time도 다시 계산해주어야 한다.
            if self.work_time_table.where(self.work_time_table['date'] == date)['end_time'].notnull()[0]:
                self.get_spend_time(date)
        
        else:
            self.work_time_table.loc[len(self.work_time_table)] =\
            [date, month, week_num, week_day, start_time, np.nan, np.nan, self.rest_time]
        
        return
        
        
    def work_end(self, end_time = None):
        if end_time == None:
            end_time = datetime.datetime.now()
        
        else:
            end_time = datetime.datetime.fromisoformat(end_time)
        
        # end_time이 24시를 넘겼을 경우에 대한 고려가 필요
        date = datetime.datetime.strftime(end_time, '%Y%m%d')
        end_time = datetime.datetime.strftime(end_time, '%Y-%m-%d %H:%M')
        
        # input
        self.work_time_table.loc[self.work_time_table['date'] == date, 'end_time'] = end_time
        
        # spend_time 계산
        self.get_spend_time(date)
        
        return

    
    def get_spend_time(self, date):
        start_time, end_time = self.work_time_table.loc[self.work_time_table['date'] == date][['start_time', 'end_time']].values[0]
        
        start_time = datetime.datetime.fromisoformat(start_time)
        end_time = datetime.datetime.fromisoformat(end_time)
        
        spend_time = end_time - start_time - datetime.timedelta(hours = 1)
        spend_time = spend_time.total_seconds()
        
        # 한시간을 안넘을 경우, - 값이 나와서 값이 정확하게 안나옴
        if spend_time >= 0:
            spend_hours = int(spend_time // 3600)
            spend_minutes = int((spend_time % 3600) // 60)

            spend_time = f'{spend_hours:02d}:{spend_minutes:02d}'
        
        else:
            spend_hours = int(abs(spend_time) // 3600)
            spend_minutes = int((abs(spend_time) % 3600) // 60)
            
            spend_time = f'-{spend_hours:02d}:{spend_minutes:02d}'
        
        # input
        self.work_time_table.loc[self.work_time_table['date'] == date, 'spend_time'] = spend_time
        
        return
        
        
    def get_total_time(self, by, target_month = None):
        # by 값에 따라 월별 / 주별 근무시간 합산 및 평균 계산
        df = self.work_time_table.copy()
        df = df.loc[df['end_time'].notnull()]
        df['spend_time_delta'] = df['spend_time'].apply(lambda x: datetime.timedelta(hours = int(x.split(':')[0]), minutes = int(x.split(':')[1])))
        
        if by == 'm':
            g_df = df.groupby(['month']).agg(sum_hour = ('spend_time_delta', 'sum'),
                                             mean_hour = ('spend_time_delta', 'mean'))
        
        elif by == 'w':
            g_df = df.groupby(['month', 'week_num']).agg(sum_hour = ('spend_time_delta', 'sum'),
                                                         mean_hour = ('spend_time_delta', 'mean'))
        
        g_df['sum_hour'] = g_df['sum_hour'].apply(self.cal_seconds_to_hour)
        g_df['mean_hour'] = g_df['mean_hour'].apply(self.cal_seconds_to_hour)
        
        return g_df
        
        
    def cal_seconds_to_hour(self, input_timedelta): # input_seconds는 datetime.timedelta 자료형
        # datetime.timedelta 자료형은 기본적으로 days, seconds, microseconds만 내부적으로 저정한다.
        h = input_timedelta.days * 24 + input_timedelta.seconds // 3600
        m = (input_timedelta.seconds % 3600) // 60
        
        t = f'{int(h):02d}:{int(m):02d}'
            
        return t
        
        
    def get_end_time(self, plan_time, start_time = None):
        if not start_time:
            start_time = datetime.datetime.now()
            today = datetime.datetime.strftime(start_time, '%Y%m%d')
        
        else:
            start_time = datetime.datetime.isoformat(start_time)
            today = datetime.datetime.strftime(start_time, '%Y%m%d')
            
        start_time = self.work_time_table.loc[self.work_time_table['date'] == today]['start_time'].values[0]
        start_time = datetime.datetime.fromisoformat(start_time)
        
        plan_hour, plan_minute = map(int, plan_time.split(':'))
        plan_time = datetime.timedelta(
                                        hours = plan_hour + self.rest_time,
                                        minutes = plan_minute
        )
        
        end_time = start_time + plan_time
        print(f"퇴근 예정 시간은 {int(end_time.hour):02d}:{int(end_time.minute)} 입니다.")
        
        return
    
    
    def get_duration(self, end_time, start_time = None):
        if not start_time:
            start_time = datetime.datetime.now()
            today = datetime.datetime.strftime(start_time, '%Y%m%d')
        
        else:
            start_time = datetime.datetime.isoformat(start_time)
            today = datetime.datetime.strftime(start_time, '%Y%m%d')
            
        start_time = self.work_time_table.loc[self.work_time_table['date'] == today]['start_time'].values[0]
        start_time = datetime.datetime.fromisoformat(start_time)
        
        end_time = datetime.datetime.fromisoformat(end_time)
        
        duration = end_time - start_time - datetime.timedelta(hours = 1)
        duration = self.cal_seconds_to_hour(duration)
        print(f"근무 예정 시간은 {duration} 입니다.")

        return

In [14]:
work_data = working_time_calculator()

In [190]:
work_data.work_time_table = df.copy()

In [192]:
date = '20240510'
start_time = '09:35'
# end_time = '16:35'

start_time = f"{date[:4]}-{date[4:6]}-{date[6:8]} {start_time}"
# end_time = f"{date[:4]}-{date[4:6]}-{date[6:8]} {end_time}"
print(start_time)
# print(end_time)

work_data.work_start(start_time)
# work_data.work_end(end_time)
display(work_data.work_time_table)

2024-05-10 09:35


,date,month,week_num,week_day,start_time,end_time,spend_time,rest_time
0,20240502,202405,18,목요일,2024-05-02 09:54,2024-05-02 17:06,06:12,1
1,20240503,202405,18,금요일,2024-05-03 08:55,2024-05-03 18:01,08:06,1
2,20240507,202405,19,화요일,2024-05-07 09:46,2024-05-07 20:05,09:19,1
3,20240508,202405,19,수요일,2024-05-08 09:26,2024-05-08 20:00,09:34,1
4,20240509,202405,19,목요일,2024-05-09 09:44,2024-05-09 16:35,05:51,1
5,20240510,202405,19,금요일,2024-05-10 09:35,NaN,NaN,1


In [193]:
work_data.get_total_time(by = 'm')

,sum_hour,mean_hour
month,,
202405,39:02,07:48


In [194]:
work_data.get_total_time(by = 'w')

sum_hour mean_hour
month  week_num                   
202405 18          14:18     07:09
       19          24:44     08:14

In [195]:
work_data.get_end_time(plan_time = '08:00')

퇴근 예정 시간은 18:35 입니다.


In [196]:
work_data.get_duration(end_time = '2024-05-10 17:30')

근무 예정 시간은 06:55 입니다.
